# Enrichment analysis workflow for molecular QTL results

For molecular QTL analysis results we obtained we'd like to see if:

1. PIP are higher on average in certain annotation groups than in the rest of genome
2. Is there an enrichment for variables both in CS and in some annotation groups
    - Specifically, whether or not there is an enrichment in the secondary CS that we capture
    
We focus only on the results that has 1 or more CS identified.

## Annotations used

We use the following types of annotations:

1. All histone marks for LCLs from Epigenome Roadmap
2. [CTCF binding sites](http://www.ebi.ac.uk/birney-srv/CTCF-QTL/)
3. POL II ChIP-seq
4. Intron and exon regions
5. Extended splice site, defined as 5bp up/down intron start site and 15bp up/down intron end site

I will first test for enrichment with the 1st CS, then the addional CS. The idea is to discover enrichment of annotations in the first CS, then also show that the 2nd CS has similar pattern of enrichment.

In [10]:
%revisions -s -n 10

Revision,Author,Date,Message
7f14848,Gao Wang,2018-07-24,Add GREGOR for matched enrichment analysis
24b7bf3,Gao Wang,2018-07-13,Update documentation
9531187,Gao Wang,2018-07-12,Add CS based enrichment
e722773,Gao Wang,2018-07-12,Update enrichment results
305751a,Gao Wang,2018-07-12,Add enrichment results


In [2]:
! sos run 20180712_Enrichment_Workflow.ipynb -h

usage: sos run 20180712_Enrichment_Workflow.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  extract_sumstats
  zscore2bed
  get_variants
  range2var_annotation
  pip_rank_test
  cs_fisher_test

Global Workflow Options:
  --y-data . (as path)
                        Y data, the phenotype file paths
  --trait VAL (required)
                        Trait name
  --cwd  path(f'{y_data:d}/{trait}_output')

                        Specify work / output directory
  --annotation-dir /home/gaow/Documents/GIT/LargeFiles/he_lab_annotations_bed (as path)
                        Path to directory of annotation
                        files
  --single-annot . (as path)
         

## Annotation input


Annotation files are in bed format (for example: `Coding_UCSC.bed`).

```
chr1    69090   70008
chr1    367658  368597
chr1    621095  622034
......
chr9    141121357       141121553
chr9    141124188       141124276
chr9    141134069       141134172
```

There are many annotations one can use. For this workflow one should prepare a list of annotations for input, like this:

```
# my annotation
Coding_UCSC
CTCF_binding
E116-DNase.macs2
E116-H2A.Z
E116-H3K27ac
E116-H3K27me3
E116-H3K36me3
E116-H3K4me1
E116-H3K4me2
E116-H3K4me3
E116-H3K79me2
E116-H3K9ac
E116-H3K9me3
E116-H4K20me1
E117-DNase.macs2
Intron_UCSC
PolII_binding
```

comment symbol `#` is allowed.

## Data input

Need to format the data to:

```
Variant_ID PIP z_score CS_ID 
```

Where `CS_ID` is 0 if variant is not in SuSiE CS, 1 if in CS 1, 2 in CS 2, etc. `Variant_ID` carries information of chrom and pos. eg, `rs10131831_chr14_20905250_G_A`

In [ ]:
[global]
# Y data, the phenotype file paths
parameter: y_data = path()
# Trait name
parameter: trait = None
# Specify work / output directory
parameter: cwd = path(f'{y_data:d}/{trait}_output')
# Path to directory of annotation files
parameter: annotation_dir = path('~/Documents/GIT/LargeFiles/lcl_annotation')
# Path to list of single annotations to use
parameter: single_annot = path() #parameter: single_annot = path("data/all_annotations.txt")
# Maximum distance to site of interest, set to eg. 100Kb or 1Mb up/downstream to start site of analysis unit
parameter: max_dist = 100000
fail_if(not y_data.is_file(), msg = 'Please provide valid ``--y-data``!')
parameter: z_score = path(f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.gz')
out_dir = f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/{z_score:bn}'.replace('.', '_')
try:
    single_anno = [f"{annotation_dir}/{x.split()[0]}.bed" for x in open(single_annot).readlines() if not x.startswith('#')]
except (FileNotFoundError, IsADirectoryError):
    single_anno = []

## Prepare annotations

Histone marks

```
sos run analysis/20180712_Enrichment_Workflow.ipynb download_histone_annotation --trait AS \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz
```

In [ ]:
[download_histone_annotation]
dataset = paths(["E116-DNase.macs2.narrowPeak.gz",
"E116-H2A.Z.narrowPeak.gz",
"E116-H3K4me1.narrowPeak.gz",
"E116-H3K4me2.narrowPeak.gz",
"E116-H3K4me3.narrowPeak.gz",
"E116-H3K9ac.narrowPeak.gz",
"E116-H3K9me3.narrowPeak.gz",
"E116-H3K27ac.narrowPeak.gz",
"E116-H3K27me3.narrowPeak.gz",
"E116-H3K36me3.narrowPeak.gz",
"E116-H3K79me2.narrowPeak.gz",
"E116-H4K20me1.narrowPeak.gz"])
input: for_each = 'dataset', concurrent = True
output: f'{annotation_dir}/{_dataset:nn}.bed'
download: dest_dir = annotation_dir, expand = True
    https://egg2.wustl.edu/roadmap/data/byFileType/peaks/consolidated/narrowPeak/{_dataset}
bash: expand = True, workdir = annotation_dir
    zcat {_dataset} | cut -f1-3 | grep -v chrM > {_output}

CTCF binding:

```
sos run analysis/20180712_Enrichment_Workflow.ipynb download_ctcf_annotation --trait AS \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz
```

In [ ]:
[download_ctcf_annotation]
output: f'{annotation_dir}/CTCF_binding.bed'
download: dest_dir = annotation_dir
    https://www.ebi.ac.uk/birney-srv/CTCF-QTL/phenotypes/bindings.csv
bash: expand = True, workdir = annotation_dir
    cut -f1-3 -d "," bindings.csv | tail -n+2 | sed 's/,/\t/g' | sed 's/^/chr/g' | grep -v chrM > {_output}

PolyII binding:

```
sos run analysis/20180712_Enrichment_Workflow.ipynb download_polII_annotation --trait AS \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz
```

Need to `liftOver`:

```
conda config --add channels bioconda
conda install ucsc-liftover
```

In [ ]:
[download_polII_annotation]
dataset = "GSM487431_GM12878_Pol2_narrowPeak.bed.gz"
output: f'{annotation_dir}/PolII_binding.bed'
download: dest_dir = annotation_dir, expand=True
    ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM487nnn/GSM487431/suppl/{dataset}
download: dest_dir = annotation_dir
    http://hgdownload.cse.ucsc.edu/goldenpath/hg18/liftOver/hg18ToHg19.over.chain.gz
bash: expand = True, workdir = annotation_dir
    zcat {dataset} | cut -f1-3  | grep -v chrM > {_output:n}.hg18.bed
    liftOver {_output:n}.hg18.bed hg18ToHg19.over.chain.gz {_output} unmapped

Extended splice sites:

```
sos run analysis/20180712_Enrichment_Workflow.ipynb extended_splice_site --trait AS \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz
```

In [ ]:
[extended_splice_site]
input: f"{annotation_dir}/Intron_UCSC.bed"
output: f"{annotation_dir}/Extended_splice_site.bed"
python: expand = "${ }", workdir = annotation_dir
    lines = [x.strip().split() for x in open(${_input:r}).readlines()]
    out = []
    for line in lines:
        out.append(f'{line[0]}\t{int(line[1])-5}\t{int(line[1])+5}')
        out.append(f'{line[0]}\t{int(line[2])-15}\t{int(line[2])+15}')
    with open(${_output:r}, 'w') as f:
        f.write('\n'.join(out))

## Prepare summary statistics file

```
sos run analysis/20180712_Enrichment_Workflow.ipynb extract_sumstats \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 
```

In [ ]:
# Extract summary stats from RDS files to plain text
[extract_sumstats_1]
input: glob.glob(f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/SuSiE_CS_[1-9]/*.rds'), group_by = 1, concurrent = True
output: f'{_input:dd}/enrichment/{_input:bn}.sumstats.gz'
R: expand = '${ }'
    dat = readRDS(${_input:r})
    pip = dat$pip
    names = names(readRDS(dat$input)[[dat$idx]]$z_score)
    zscore = readRDS(dat$input)[[dat$idx]]$z_score
    cs_id = rep(0, length(pip))
    for (i in 1:length(dat$sets$cs)) {
        cs_id[dat$sets$cs[[i]]] = dat$sets$cs_index[i]
    }
    write.table(cbind(names,pip,zscore,cs_id), gzfile(${_output:r}), quote=FALSE, col.names=FALSE, row.names=FALSE, sep="\t")

# Consolidate results to one file
[extract_sumstats_2]
output: f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.gz'
bash: expand = True
    zcat {_input} | gzip --best > {_output}
_input.zap()

```
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | wc -l
1667666
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 1 | wc -l
46659
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 2 | wc -l
2212
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 3 | wc -l
152
```

So we have total of 1667666 variants, 46659 in CS 1, 2212 in CS 2.

## Convert variants from summary statistics to bed format

In [ ]:
# Auxiliary step to get variant in bed format based on variant ID in z-score file
[zscore2bed_1]
parameter: in_file = path()
parameter: chr_prefix = ""
input: in_file
output: f'{_input:n}.bed.unsorted'
R: expand = "${ }", docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    library(readr)
    library(stringr)
    library(dplyr)
    var_file <- ${_input:r}
    out_file <- ${_output:r}

    variants <- read_tsv(var_file)
    colnames(variants) = c('variant', 'pip', 'zscore', 'cs')
    var_info <- str_split(variants$variant, "_")
    variants <- mutate(variants, chr = paste0("${chr_prefix}", sapply(var_info, function(x){x[2]})), 
                                 pos = sapply(var_info, function(x){x[3]})) %>%
                mutate(start = as.numeric(pos), stop=as.numeric(pos)  + 1) %>%
                select(chr, start, stop, variant)
    options(scipen=1000) # So that positions are always fully written out)
    write.table(variants, file=out_file, quote=FALSE, col.names=FALSE, row.names=FALSE, sep="\t")

[zscore2bed_2]
output: f'{_input:n}'
bash: expand = True, docker_image = 'gaow/atac-gwas', workdir = cwd
     sort-bed {_input} > {_output}
_input.zap()

[get_variants: provides = '{data}.bed']
output: f'{data}.bed'
sos_run('zscore2bed', in_file = f'{_output:n}.gz')

## Apply ranged based annotations

```
sos run analysis/20180712_Enrichment_Workflow.ipynb range2var_annotation \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Get variants in data that falls in target region
[range2var_annotation_1]
depends: f'{z_score:n}.bed'
input: set(paths(single_anno)), group_by = 1, concurrent = True
output: f'{out_dir}/{_input:bn}.{z_score:bn}.bed'
bash: expand = True, docker_image = 'gaow/atac-gwas', workdir = cwd, volumes = f'{annotation_dir}:{annotation_dir}'
    bedops -e {z_score:n}.bed {_input} > {_output}

In [ ]:
# Make binary annotation file
[range2var_annotation_2]
depends: z_score
input: group_by = 1, concurrent = True
output: f'{_input:n}.gz'
R: expand = "${ }", docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    library(readr)
    library(dplyr)
    library(stringr)

    variant_tsv <- ${z_score:r}
    annotation_var_bed <- ${_input:r}
    annot_name <- ${_input:bnr} %>% str_replace(paste0(".",${z_score:bnr}), "")
    out_name <- ${_output:r}

    vars <- read_tsv(variant_tsv)[,1]
    annot_vars = read_tsv(annotation_var_bed, col_names=FALSE)
    names(vars) <- "SNP"
    vars <- vars %>%
            mutate(annot_d = case_when(SNP %in% annot_vars$X4 ~ 1,
                                                        TRUE ~ 0))
    names(vars)[2] <- annot_name
    write.table(vars, file=gzfile(out_name),
                col.names=TRUE, row.names=FALSE, sep="\t", quote=FALSE)

## Matched enrichment analysis via GREGOR

To properly perform enrichment analysis we want to match the control SNPs with the SNPs of interest -- that is, SNPs inside CS -- in terms of LD, distance to nearest gene and MAF. The [GREGOR](http://csg.sph.umich.edu/GREGOR/index.php/site/download) software can generate list of matched SNPs. I will use SNPs inside CS as input and expect a list of output SNPs matching these inputs.

GREGOR is release under University of Michigan license so I'll not make it into a docker image. So path to GREGOR directory is required. Also we need reference files, prepared by:

```
cat \
    GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz.part.00 \
    GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz.part.01 \
    > GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz
tar zxvf GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz
```

MD5SUM check:

```
AFR.part.0	( MD5: 9926904128dd58d6bf1ad4f1e90638af )
AFR.part.1	( MD5: c1d30aff89a584bfa8c1fa1bdc197f21 )
```

The input file can be a list of `rs` ID's. In `SuSiE_loci.sumstats.gz` we do have this information. 

```
sos run analysis/20180712_Enrichment_Workflow.ipynb gregor \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
[gregor_1 (make SNP index)]
input: z_score
output: f'{_input:nn}.rsid.txt', f'{_input:nn}.annotations.list'
bash: expand = '${ }'
    zcat ${_input} | cut -f 2,3 -d "_" | sed 's/_/:/g' > ${_output[0]}

with open(_output[1], 'w') as f:
    f.write('\n'.join(single_anno))

[gregor_2 (make configuration file)]
parameter: gregor_db = path('~/GIT/LargeFiles/GREGOR_DB')
parameter: pop = 'AFR'
parameter: ncpu = 10
output: f'{_input[0]:nn}.gregor.conf'
report: output = f'{_output}', expand = True
    ##############################################################################
    # CHIPSEQ ENRICHMENT CONFIGURATION FILE
    # This configuration file contains run-time configuration of
    # CHIP_SEQ ENRICHMENT
    ###############################################################################
    ## KEY ELEMENTS TO CONFIGURE : NEED TO MODIFY
    ###############################################################################
    INDEX_SNP_FILE = {_input[0]}
    BED_FILE_INDEX = {_input[1]} 
    REF_DIR = {gregor_db}
    R2THRESHOLD = 0.7 ## must be greater than 0.7
    LDWINDOWSIZE = 10000 ## must be less than 1MB; these two values define LD buddies
    OUT_DIR = {_output:nn}_gregor_output
    MIN_NEIGHBOR_NUM = 10 ## define the size of neighborhood
    BEDFILE_IS_SORTED = true  ## false, if the bed files are not sorted
    POPULATION = {pop}  ## define the population, you can specify EUR, AFR, AMR or ASN
    TOPNBEDFILES = 2 
    JOBNUMBER = {ncpu}
    ###############################################################################
    #BATCHTYPE = mosix ##  submit jobs on MOSIX
    #BATCHOPTS = -E/tmp -i -m2000 -j10,11,12,13,14,15,16,17,18,19,120,122,123,124,125 sh -c
    ###############################################################################
    #BATCHTYPE = slurm   ##  submit jobs on SLURM
    #BATCHOPTS = --partition=broadwl --account=pi-mstephens --time=0:30:0
    ###############################################################################
    BATCHTYPE = local ##  run jobs on local machine

bash: expand = True
    sed -i '/^$/d' {_output}

GREGOR is written in `perl`. Some libraries are required to run GREGOR:

```
sudo apt-get install libdbi-perl libswitch-perl libdbd-sqlite3-perl
```

In [ ]:
[gregor_3 (run gregor)]
parameter: gregor_path = path('/opt/GREGOR')
output: f'{_input:nn}_gregor_output/StatisticSummaryFile.txt'
bash: expand = True
    perl {gregor_path}/script/GREGOR.pl --conf {_input} && touch {_output}

[gregor_4 (format output)]
output: f'{_input:n}.csv'
bash: expand = True
    sed 's/\t/,/g' {_input} > {_output}

In [7]:
dat = read.table('/home/gaow/Documents/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/enrichment/SuSiE_loci_gregor_output/StatisticSummaryFile.csv', head=T, sep=',')
dat[order(dat$PValue),]

,Bed_File,InBed_Index_SNP,ExpectNum_of_InBed_SNP,PValue
2,POLII_DeMayo.bed,148424,66768.2288,0.000000e+00
4,TSS_Hoffman.bed,105040,51686.5347,0.000000e+00
7,DGF_ENCODE.bed,352229,304847.0034,0.000000e+00
9,h3k4me3-final-C-48h.bed,95058,51763.8750,0.000000e+00
12,CN_range_anno.bed,120947,108629.8275,0.000000e+00
13,h3k4me1-final-C-48h.bed,208171,156068.2213,0.000000e+00
15,UTR_3_UCSC.bed,62993,38469.2608,0.000000e+00
16,NSC_range_anno.bed,112092,98634.1120,0.000000e+00
17,Enhancer_Andersson.bed,23736,16741.3500,0.000000e+00
18,E8_TCM_D_48h.bed,159330,118319.0294,0.000000e+00


## Unmatched enrichment analysis

In Li 2016 manuscript the analysis was not matched by LD, MAF and TSS for sQTL. The only constraint for control SNPs was that they should be within intron clusters. To focus analysis on these SNPs:

```
sos run analysis/20180712_Enrichment_Workflow.ipynb overlap_cluster \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS
```

In [ ]:
# Overlap z-scores with intron clusters
[overlap_cluster]
input: f"{z_score:n}.bed"
output: f'{out_dir}/{y_data:bnn}.{z_score:bn}.overlap.bed'
bash: expand = True, docker_image = 'gaow/atac-gwas', workdir = cwd, volumes = [f'{annotation_dir}:{annotation_dir}', f"{y_data:d}:{y_data:d}"]
    bedops -e {_input} <(zcat {y_data} | cut -f1-3 | tail -n+2) > {_output}

Test for larger PIP in annotation vs outside it. 

```
sos run analysis/20180712_Enrichment_Workflow.ipynb pip_rank_test \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Test if PIP is larger in annotations
[pip_rank_test_1]
depends: z_score, f'{out_dir}/{y_data:bnn}.{z_score:bn}.overlap.bed'
input_files = [f'{out_dir}/{value:bn}.{z_score:bn}.gz' for value in paths(single_anno)]
input: input_files, group_by = 1, concurrent = True
output: f'{_input:n}.{step_name}.csv'
R: expand = '${ }', docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    set.seed(1)
    library(readr)
    library(dplyr)
    variants <- read_tsv(${z_score:r}, col_names=FALSE)
    colnames(variants) = c('SNP', 'PIP', 'Z', 'CS')
    annotation <- read_tsv(${_input:r}, col_names=TRUE)
    name = colnames(annotation)[2]
    colnames(annotation) = c('SNP', 'GROUP')
    cluster_subset <- read_tsv('${out_dir}/${y_data:bnn}.${z_score:bn}.overlap.bed', col_names=FALSE)
    colnames(cluster_subset) = c("CHR", "START", "END", "SNP")
    # add two random groupings
    annotation$RAND_1 = sample(annotation$GROUP)
    annotation$RAND_2 = sample(annotation$GROUP)
    variants = inner_join(variants, annotation, cluster_subset, by = "SNP")
    if (length(unique(variants$GROUP)) == 1) {
      write(paste(name, NA, NA, NA, sep=','), file = ${_output:r})
    } else {
    test = wilcox.test(PIP ~ GROUP, data=variants)
    c_1 = wilcox.test(PIP ~ RAND_1, data=variants)
    c_2 = wilcox.test(PIP ~ RAND_2, data=variants)
    write(paste(name, test$p.value, c_1$p.value, c_2$p.value, sep=','), file = ${_output:r})
    }

# Consolidate results to one file
[pip_rank_test_2, cs_fisher_test_2]
output: f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.{step_name}.csv'
bash: expand = True
    cat {_input} > {_output}
_input.zap()

In [5]:
dat = read.table('/home/gaow/Documents/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/enrichment/SuSiE_loci.sumstats.pip_rank_test_2.csv', head=F, sep=',')
colnames(dat) = c('annotation', 'enrichment', 'random_1', "random_2")
dat[order(dat[,2]),]

,annotation,enrichment,random_1,random_2
15,Intron_UCSC,0.000000e+00,0.694988243,0.58010513
11,E116-H3K79me2,2.837897e-161,0.283023549,0.57836372
1,Coding_UCSC,5.430452e-150,0.806499327,0.86154126
17,Extended_splice_site,2.440152e-44,0.462324059,0.15302982
2,CTCF_binding,2.444353e-28,0.206441433,0.41081027
9,E116-H3K4me2,2.613140e-09,0.193819551,0.82515639
4,E116-H2A.Z,7.724020e-07,0.006410806,0.19196325
5,E116-H3K27ac,1.990628e-03,0.938771084,0.07614115
7,E116-H3K36me3,2.398294e-03,0.501105064,0.16285261
10,E116-H3K4me3,6.046327e-03,0.911334021,0.66381686


Test for enrichment of annotation in CS.

```
sos run analysis/20180712_Enrichment_Workflow.ipynb cs_fisher_test \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Test if CS is enriched with annotations
[cs_fisher_test_1]
depends: z_score, f'{out_dir}/{y_data:bnn}.{z_score:bn}.overlap.bed'
input_files = [f'{out_dir}/{value:bn}.{z_score:bn}.gz' for value in paths(single_anno)]
input: input_files, group_by = 1, concurrent = True
output: f'{_input:n}.{step_name}.csv'
R: expand = '${ }', docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    run_test = function(dat) {
        d1 = dat
        d1$CS[which(d1$CS>0)] = 1
        d1 = table(d1)
        test.d1 = fisher.test(d1)
        # test for non-first CS only
        d2 = dat[which(dat$CS != 1),]
        d2$CS[which(d2$CS>0)] = 1
        d2 = table(d2)
        test.d2 = fisher.test(d2)
        return(c(test.d1$p.value, test.d2$p.value, test.d1$estimate, test.d2$estimate))
    }
    set.seed(1)
    library(readr)
    library(dplyr)
    variants <- read_tsv(${z_score:r}, col_names=FALSE)
    colnames(variants) = c('SNP', 'PIP', 'Z', 'CS')
    annotation <- read_tsv(${_input:r}, col_names=TRUE)
    name = colnames(annotation)[2]
    colnames(annotation) = c('SNP', 'GROUP')
    cluster_subset <- read_tsv('${out_dir}/${y_data:bnn}.${z_score:bn}.overlap.bed', col_names=FALSE)
    colnames(cluster_subset) = c("CHR", "START", "END", "SNP")
    # add two random groupings
    annotation$RAND_1 = sample(annotation$GROUP)
    annotation$RAND_2 = sample(annotation$GROUP)
    variants = inner_join(variants, annotation, cluster_subset, by = "SNP")[,4:7]
    if (length(unique(variants$GROUP)) == 1) {
      write(paste(name, NA, NA, NA, NA, NA, NA, NA, NA, sep=','), file = ${_output:r})
    } else {
    # test against all CS
    test = run_test(variants[,c('CS', 'GROUP')])
    ctrl_1 = run_test(variants[,c('CS', 'RAND_1')])
    ctrl_2 = run_test(variants[,c('CS', 'RAND_2')])
    write(paste(name, test[1], test[3], test[2], test[4], ctrl_1[1], ctrl_1[2], ctrl_2[1], ctrl_2[2], sep=','), file = ${_output:r})
    }

In [4]:
dat = read.table('/home/gaow/Documents/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/enrichment/SuSiE_loci.sumstats.cs_fisher_test_2.csv', head=F, sep=',')
colnames(dat) = c('annotation', 'enrichment CS>0', 'odds ratio CS>0', 'enrichment CS>1', "odds ratio CS>1", "random_1 CS>0", "random_1 CS>1", "random_2 CS>0", "random_2 CS>1")
dat[order(dat[,2]),]

,annotation,enrichment CS>0,odds ratio CS>0,enrichment CS>1,odds ratio CS>1,random_1 CS>0,random_1 CS>1,random_2 CS>0,random_2 CS>1
15,Intron_UCSC,0.000000e+00,1.899154,4.251291e-193,2.0728023,0.06105334,0.450992553,0.172685600,0.43010819
17,Extended_splice_site,6.102566e-102,1.966429,1.574085e-03,1.5802937,0.81700532,0.468165390,0.159701645,0.58754997
1,Coding_UCSC,3.084959e-86,1.280004,1.397209e-04,0.7860334,0.07515717,0.415979703,0.595173143,0.69584542
2,CTCF_binding,1.177969e-18,1.287152,6.533736e-38,3.1884346,0.00637921,0.002802365,0.380667199,0.06829601
5,E116-H3K27ac,1.754503e-16,0.000000,2.760625e-01,0.0000000,0.04124679,1.000000000,0.698024012,0.41264853
16,PolII_binding,5.780630e-04,0.346048,6.388968e-01,0.0000000,0.69522884,1.000000000,0.003600851,1.00000000
12,E116-H3K9ac,5.873009e-03,30.279181,1.000000e+00,0.0000000,1.00000000,1.000000000,1.000000000,1.00000000
11,E116-H3K79me2,1.645288e-02,1.569635,6.348330e-01,0.0000000,0.76533829,1.000000000,0.008634295,0.13203453
4,E116-H2A.Z,1.309249e-01,2.125093,1.000000e+00,0.0000000,0.16042764,1.000000000,0.765390556,1.00000000
7,E116-H3K36me3,1.000000e+00,0.000000,1.000000e+00,0.0000000,1.00000000,1.000000000,1.000000000,1.00000000


There is more consistency between matched and unmatched PIP analysis. But in general, most annotations are significant.